# Import Libs

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" #If the line below doesn't work, uncomment this line (make sure to comment the line below); it should help.
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
'''
Numbers for "os.environ['TF_CPP_MIN_LOG_LEVEL']": 
0 = all messages are logged (default behavior)
1 = INFO messages are not printed
2 = INFO and WARNING messages are not printed
3 = INFO, WARNING, and ERROR messages are not printed
'''

import tensorflow
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # Ignore Pandas warnings of deprecation
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn' | Disable warnings
import numpy as np
from time import time
from keras.layers import Dense, Dropout, SimpleRNN, RNN, LSTM, GRU
from keras import Sequential, models
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from termcolor import colored
import matplotlib.pyplot as plt
!pip install tensorflow_model_optimization
import tensorflow_model_optimization as tfmot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 238 kB 14.8 MB/s 


# Import other Python files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/')
!cp -r "/content/drive/My Drive/Colab Notebooks/unsw_processing.ipynb" '/content/'
!cp -r "/content/drive/My Drive/Colab Notebooks/results_visualization.ipynb" '/content/'

!pip install import_ipynb
import import_ipynb

from unsw_processing import unsw_encoding
from results_visualization import print_results


print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))
print(tensorflow.test.gpu_device_name())


Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 14.0 MB/s 
importing Jupyter notebook from unsw_processing.ipynb
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
importing Jupyter notebook from results_visualization.ipynb
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
^C
Num GPUs Available:  0



# Training

In [ ]:
# Values for later use
csv_values = ['epochs', 'acc', 'loss', 'val_acc', 'val_loss',
              'loss_fct', 'optimizer', 'activation_fct',
              'layer_nb', 'unit_nb', 'batch_size', 'dropout', 'cell_type',
              'encoder']

csv_best_res = ['param', 'value', 'min_mean_val_loss']

train_params = {'epochs': 100, 
          'loss_fct': 'mse', 
          'optimizer': 'rmsprop',
          'activation_fct': 'tanh', 
          'layer_nb': 1, 'unit_nb': 128, 
          'batch_size': 1024, 'dropout': 0.4,
          'encoder': 'labelencoder',
          'shuffle': True}


# ***** VARIABLE PARAMETERS *****
# 'encoder': ['standardscaler', 'labelencoder', 'minmaxscaler01', 'minmaxscaler11', 'ordinalencoder'],
# 'optimizer': ['adam', 'sgd', 'rmsprop', 'nadam', 'adamax', 'adadelta'],
# 'activation_fct': ['sigmoid', 'softmax', 'relu', 'tanh']
# 'layer_nb': [1, 2, 3, 4]
# 'unit_nb': [4, 8, 32, 64, 128, 256]
# 'dropout': [0.1, 0.2, 0.3, 0.4]
# 'batch_size': [512, 1024, 2048]


tf_model_path = '/content/content/tf_models'
res_path = "/content/drive/My Drive/Colab Notebooks/results/"

# Name for results file
res_name = train_params['loss_fct'] + '_' + train_params['optimizer'] + '_' +\
        train_params['activation_fct'] + '_' + str(train_params['layer_nb']) + '_' +\
        str(train_params['unit_nb']) + '_' + str(train_params['batch_size']) + '_' +\
        str(train_params['dropout']) + '_' + "RNN" + '_' +\
        train_params['encoder'] + '_' + str(time())

In [ ]:
# Encode dataset and return : x_train, x_test, y_train, y_tests
def load_data():
    x_train, x_test, y_train, y_test = unsw_encoding(train_params)

    # Reshape the inputs in the accepted model format
    x_train = np.array(x_train).reshape([-1, x_train.shape[1], 1])
    x_test = np.array(x_test).reshape([-1, x_test.shape[1], 1])
    return x_train, x_test, y_train, y_test

In [ ]:
# Create and train a model
def train_model(x_train, x_test, y_train, y_test):
    cell = SimpleRNN #RNN
  
    # Create a Sequential layer, one layer after the other
    model = Sequential()
    # If there is more than 1 layer, the first must return sequences
    for _ in range(train_params['layer_nb']-1):
        model.add(cell(units=train_params['unit_nb'],
                    input_shape=(x_train.shape[1:]), return_sequences=True))
        model.add(Dropout(rate=train_params['dropout']))

    # If there is only 1 layer, it must not return sequences
    if(train_params['layer_nb'] == 1):
        model.add(cell(units=train_params['unit_nb'], input_shape=x_train.shape[1:]))
        model.add(Dropout(rate=train_params['dropout']))
    else:  # If there is more than 1, the following must not return sequences
        model.add(cell(units=train_params['unit_nb']))
        model.add(Dropout(rate=train_params['dropout']))
    # Outputs layer
    model.add(Dense(units=y_train.shape[1],
                    activation=train_params['activation_fct']))

    model.compile(loss=train_params['loss_fct'], optimizer=train_params['optimizer'],
                metrics=['accuracy'])

    model.summary()

    hist = model.fit(x_train, y_train, train_params['batch_size'], train_params['epochs'],
                    verbose=1, shuffle=train_params['shuffle'],
                    validation_data=(x_test, y_test), callbacks=None)
    
    save_model = ModelCheckpoint(filepath=tf_model_path + res_name,
                                        monitor='val_accuracy', save_best_only=True)
    callbacks = [save_model]
    model.save(tf_model_path)

    print_results(train_params, model, x_train, x_test, y_train, y_test)

    return hist

In [ ]:
if __name__ == "__main__":
    print("Loading Data")
    x_train, x_test, y_train, y_test = load_data()
    print("Training Model")
    history = train_model(x_train, x_test, y_train, y_test)
    
    # --- SAVE RESULTS --- #
    
    epochs = len(history.history['accuracy'])
    df = pd.DataFrame(columns=csv_values)

    try:
      for epoch in range(epochs):
        df = df.append({'epochs': epoch,
                        'acc':  history.history['accuracy'][epoch],
                        'loss': history.history['loss'][epoch],
                        'val_acc': history.history['val_accuracy'][epoch],
                        'val_loss': history.history['val_loss'][epoch],
                        'loss_fct': train_params['loss_fct'],
                        'optimizer': train_params['optimizer'],
                        'activation_fct': train_params['activation_fct'],
                        'layer_nb': train_params['layer_nb'],
                        'unit_nb': train_params['unit_nb'],
                        'batch_size': train_params['batch_size'],
                        'dropout': train_params['dropout'],
                        'cell_type': "RNN",
                        'encoder': train_params['encoder']},
                    ignore_index=True)
    except Exception as e:
      print("Could not append to df due to {}".format(colored(e, 'red')))
    
    res_name = train_params['loss_fct'] + '_' + train_params['optimizer'] + '_' +\
        train_params['activation_fct'] + '_' + str(train_params['layer_nb']) + '_' +\
        str(train_params['unit_nb']) + '_' + str(train_params['batch_size']) + '_' +\
        str(train_params['dropout']) + '_' + "RNN" + '_' +\
        train_params['encoder'] + '_' + str(time())
        
    # Errors occur without this
    if not os.path.exists(res_path):
        os.makedirs(res_path)
  
    full_res_path = res_path + "model_full_results_" + res_name + ".csv"
    df.to_csv(full_res_path, index=False)


In [ ]:
# --- Charts --- # 
    # Accuracy Chart
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0.5, 1])
    plt.xlim([0, epochs])
    plt.legend(loc='lower right')
    plt.show()

    # Loss Chart
    plt.plot(history.history['loss'], label='loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.ylim([0.01, 0.1])
    plt.xlim([0, epochs])
    plt.legend(loc='lower right')
    plt.show()

# Save models to Drive

In [ ]:
!cp -r "/content/content/tf_models" "/content/drive/My Drive/Colab Notebooks/"

# Quantize Model

In [ ]:
from google.colab import drive
import sys
import tensorflow as tf
from keras import Sequential, models
!pip install tensorflow_model_optimization
import tensorflow_model_optimization as tfmot


# RUN PREVIOUS CODE BLOCK BEFORE RUNNING THIS
# THIS CODE QUANTIZES THE CURRENT SAVED MODEL IN THE DRIVE FOLDER

# Get files from Drive
drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/Colab Notebooks/')
!cp -r "/content/drive/My Drive/Colab Notebooks/tf_models/" '/content/'

# Normal Model without Quantization
loaded_model = models.load_model('/content/tf_models')
print(loaded_model.summary())

# Helper function uses `quantize_annotate_layer` to annotate that only the Dense layers should be quantized.
def apply_quantization_to_dense(layer):
  if isinstance(layer, tf.keras.layers.Dense):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  return layer

# Use `tf.keras.models.clone_model` to apply `apply_quantization_to_dense` to the layers of the model.
annotated_model = tf.keras.models.clone_model(
    loaded_model,
    clone_function=apply_quantization_to_dense,
)

# Now that the Dense layers are annotated, `quantize_apply` actually makes the model quantization aware.
quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)
print(quant_aware_model.summary())
loaded_model.compile(loss=train_params['loss_fct'], optimizer=train_params['optimizer'],
                metrics=['accuracy'])


quant_aware_model.save("/content/quant_model")
!cp -r "/content/quant_model/" '/content/drive/My Drive/Colab Notebooks/'